In [383]:
import numpy as np
import pandas as pd

In [384]:
np.random.seed(0)

In [385]:
threshold_min, threshold_max, threshold_delta = 0., 1., 1e-1

In [386]:
def bayesian_update(priors):
    if np.sum(priors) == 0:
        return np.zeros_like(priors)
    return priors / np.sum(priors)

In [387]:
def manipulation_thresholds(thresholds, priors, c):
    if np.sum(priors) == 0:
        return thresholds
    return np.maximum(0, thresholds - (bayesian_update(priors) / c))

In [388]:
def balance_priors(priors, random=False):
    total = np.sum(priors)
    if total == 1:
        return priors
    indices = priors == 0
    remainder = 1 - total
    if random:
        p = np.random.rand(indices.sum())
        p = (p / p.sum()) * remainder
    else:
        p = remainder / indices.sum()
    priors[indices] = p
    return priors

In [389]:
def accuracy_loss(thresholds, x_manipulation, threshold_true):
    losses = []
    for i in range(len(thresholds)):
        if thresholds[i] < threshold_true:
            loss = (threshold_true - x_manipulation[i]) / (threshold_max - threshold_min)
        else:
            loss = np.abs(x_manipulation[i] - threshold_true) / (threshold_max - threshold_min)
        losses.append(loss)
    return np.array(losses)

In [390]:
thresholds = np.arange(threshold_min+threshold_delta, threshold_max, threshold_delta).round(4)

priors = np.zeros_like(thresholds)
priors[4] = 0.05
priors[5] = 0.3
priors[6] = 0.3
priors[7] = 0.1

balance_priors(priors, random=True)
print(np.sum(priors))
assert np.sum(priors) == 1

pd.DataFrame({"threshold": thresholds, "priors": priors}).round(3).T

1.0


,0,1,2,3,4,5,6,7,8
threshold,0.100,0.200,0.300,0.400,0.50,0.6,0.7,0.8,0.900
priors,0.048,0.063,0.053,0.048,0.05,0.3,0.3,0.1,0.037


In [391]:
# import itertools
# L = [1,2,3,4,5]

# threshold_partitions = []
# for L5 in itertools.permutations(L, 5):
#     threshold_partition = []
#     for part in itertools.batched(L5, 3):
#         threshold_partition.append(part)
#     threshold_partitions.append(threshold_partition)
# threshold_partitions

In [392]:
c = 1
threshold_true = 0.5
# partitions = [[0,2,4,6,8], [1,3,5,7]]
partitions = [[i] for i in range(9)]

In [393]:
for i, partition in enumerate(partitions):
    threshold_p = thresholds[partition]
    priors_p = priors[partition]
    print(f"Partition {i}")
    print(f"Thresholds               : {threshold_p.round(4)}")
    print(f"Priors                   : {priors_p.round(4)}")
    print(f"Priors (Bayesian Update) : {bayesian_update(priors_p).round(4)}")
    print(f"Manipulation Threshold   : {manipulation_thresholds(threshold_p, priors_p, c).round(4)}")
    x_manipulation_p = manipulation_thresholds(threshold_p, priors_p, c)
    acc_loss = accuracy_loss(threshold_p, x_manipulation_p, threshold_true)
    print(f"Accuracy Loss            : {acc_loss.round(4)}")
    print(f"Partition Accuracy Loss  : {np.mean(acc_loss):.4f}")
    print()

Partition 0
Thresholds               : [0.1]
Priors                   : [0.0484]
Priors (Bayesian Update) : [1.]
Manipulation Threshold   : [0.]
Accuracy Loss            : [0.5]
Partition Accuracy Loss  : 0.5000

Partition 1
Thresholds               : [0.2]
Priors                   : [0.0631]
Priors (Bayesian Update) : [1.]
Manipulation Threshold   : [0.]
Accuracy Loss            : [0.5]
Partition Accuracy Loss  : 0.5000

Partition 2
Thresholds               : [0.3]
Priors                   : [0.0531]
Priors (Bayesian Update) : [1.]
Manipulation Threshold   : [0.]
Accuracy Loss            : [0.5]
Partition Accuracy Loss  : 0.5000

Partition 3
Thresholds               : [0.4]
Priors                   : [0.048]
Priors (Bayesian Update) : [1.]
Manipulation Threshold   : [0.]
Accuracy Loss            : [0.5]
Partition Accuracy Loss  : 0.5000

Partition 4
Thresholds               : [0.5]
Priors                   : [0.05]
Priors (Bayesian Update) : [1.]
Manipulation Threshold   : [0.]
Accura